In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [3]:
!pip install --quiet bitsandbytes
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece
# !pip install flash-attn --no-build-isolation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 25.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 9.4 MB/s eta 0:00:00:00:01


In [4]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes==0.45.0


Found existing installation: bitsandbytes 0.45.3
Uninstalling bitsandbytes-0.45.3:
  Successfully uninstalled bitsandbytes-0.45.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.3 MB/s eta 0:00:00:00:0100:01


In [5]:
!pip install --upgrade crewai langchain pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 83.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
!pip uninstall crewai_tools -y


Found existing installation: crewai-tools 0.1.6
Uninstalling crewai-tools-0.1.6:
  Successfully uninstalled crewai-tools-0.1.6


In [6]:
!pip install --upgrade crewai crewai-tools embedchain langchain-openai


  Using cached embedchain-0.1.127-py3-none-any.whl.metadata (9.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 14.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install crewai_tools==0.1.6

  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
  Using cached embedchain-0.1.126-py3-none-any.whl.metadata (9.3 kB)
  Using cached embedchain-0.1.125-py3-none-any.whl.metadata (9.3 kB)
  Using cached embedchain-0.1.124-py3-none-any.whl.metadata (9.3 kB)
  Using cached embedchain-0.1.123-py3-none-any.whl.metadata (9.3 kB)
  Using cached embedchain-0.1.122-py3-none-any.whl.metadata (9.3 kB)
  Using cached embedchain-0.1.121-py3-none-any.whl.metadata (9.3 kB)
  Using cached embedchain-0.1.120-py3-none-any.whl.metadata (9.3 kB)
INFO: pip is still looking at multiple versions of em

In [9]:
!pip install --upgrade tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.105.0 requires chromadb>=0.5.23, but you have chromadb 0.4.24 which is incompatible.


In [10]:
import os
import torch
import json
import logging
from torch import bfloat16
from crewai import Agent, Task, Crew, Process
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from IPython.display import Markdown
import re
from litellm import completion  # Fix: Use LiteLLM for inference



In [11]:
logging.basicConfig(level=logging.DEBUG)


In [40]:
# Load LLaMA model for parameter extraction
model_id_llama = "alokabhishek/Llama-2-7b-chat-hf-bnb-8bit"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id_llama, use_fast=True)
model_llama = AutoModelForCausalLM.from_pretrained(model_id_llama, device_map="auto")
pipe_llama = pipeline(model=model_llama, tokenizer=tokenizer_llama, task='text-generation')

# Fix: Ensure the HuggingFacePipeline is properly configured
llm = HuggingFacePipeline(
    pipeline=pipe_llama,
    model_kwargs={"temperature": 0.7, "max_length": 512}  # Set generation params if needed
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [110]:
from crewai import LLM

llm = LLM(
    model="huggingface/alokabhishek/Llama-2-7b-chat-hf-bnb-8bit",
    base_url = "https://api-inference.huggingface.co/models/alokabhishek/Llama-2-7b-chat-hf-bnb-8bit"
)

In [111]:
# Load ProtGPT2 model for protein sequence generation
protgpt2_pipeline = pipeline('text-generation', model="nferruz/ProtGPT2")


Device set to use cuda:0


In [112]:
# Define Parameter Extraction Schema
class ParameterExtractionInput(BaseModel):
    user_prompt: str

In [113]:
class ParameterExtractionOutput(BaseModel):
    context_sequence: str = Field(default="MKVSA", description="User-provided starting sequence")
    max_length: int = Field(default=100, description="Max sequence length")
    do_sample: bool = Field(default=True, description="Enable sampling")
    top_k: int = Field(default=950, description="Top-k filtering")
    repetition_penalty: float = Field(default=1.2, description="Repetition penalty")
    num_return_sequences: int = Field(default=10, description="Number of sequences to generate")
    eos_token_id: int = Field(default=0, description="End-of-sequence token ID")


In [114]:
def extract_parameters(user_prompt: str) -> dict:
    """Extracts structured parameters for protein sequence generation."""
    system_prompt = (
        "Extract the following parameters from the given user prompt and return them strictly in valid JSON format:\n"
        "{\n"
        '  "context_sequence": "string",\n'
        '  "max_length": "integer",\n'
        '  "do_sample": "boolean",\n'
        '  "top_k": "integer",\n'
        '  "repetition_penalty": "float",\n'
        '  "num_return_sequences": "integer",\n'
        '  "eos_token_id": "integer"\n'
        "}"
    )
    input_text = f"{system_prompt}\nUser Prompt: {user_prompt}"
    response = pipe_llama(input_text)[0]["generated_text"]

    try:
        json_match = re.search(r"\{.*\}", response, re.DOTALL)
        extracted_params = json.loads(json_match.group()) if json_match else {}
        extracted_params = ParameterExtractionOutput.parse_obj(extracted_params).dict()
    except Exception as e:
        logging.warning(f"Error parsing LLM response: {e}. Using default values.")
        extracted_params = ParameterExtractionOutput().dict()

    return extracted_params

In [115]:
# Define Sequence Generation Schema
class SequenceGenerationInput(BaseModel):
    context_sequence: str = Field(description="Starting protein sequence")
    max_length: int = Field(description="Max sequence length")
    do_sample: bool = Field(description="Enable sampling")
    top_k: int = Field(description="Top-k filtering")
    repetition_penalty: float = Field(description="Repetition penalty")
    num_return_sequences: int = Field(description="Number of sequences to generate")
    eos_token_id: int = Field(description="End-of-sequence token ID")


In [116]:
def generate_sequence(params: dict) -> list:
    """Generates protein sequences based on extracted parameters."""
    logging.info(f"Generating sequences with parameters: {params}")
    sequences = protgpt2_pipeline(
        params["context_sequence"],
        max_length=params["max_length"],
        do_sample=params["do_sample"],
        top_k=params["top_k"],
        repetition_penalty=params["repetition_penalty"],
        num_return_sequences=params["num_return_sequences"],
        eos_token_id=params["eos_token_id"]
    )
    return [seq["generated_text"] for seq in sequences]


In [117]:
# Define Agents
param_extraction_agent = Agent(
    role="Parameter Extraction Specialist",
    goal="Extract structured parameters for protein generation",
    backstory="Expert in translating user intent into precise model parameters.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

generation_agent = Agent(
    role="Protein Sequence Generator",
    goal="Generate protein sequences based on structured parameters.",
    backstory="A specialist in protein language modeling, capable of generating realistic sequences.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [118]:
# Define Tools
from crewai.tools import BaseTool

class ParameterExtractionTool(BaseTool):
    name: str = "Parameter Extraction Tool"
    description: str = "Extracts structured parameters for ProtGPT2 from a user prompt."

    def _run(self, user_prompt: str) -> dict:
        return extract_parameters(user_prompt)

class SequenceGenerationTool(BaseTool):
    name: str = "Sequence Generation Tool"
    description: str = "Generates protein sequences based on extracted parameters."

    def _run(self, params: dict) -> list:
        return generate_sequence(params)


In [119]:
extract_tool = ParameterExtractionTool()
generate_tool = SequenceGenerationTool()


In [120]:
# Define Tasks
param_extraction_task = Task(
    description="Extracts necessary parameters from the user input.",
    expected_output="A structured parameter list for sequence generation.",
    tools=[extract_tool],
    agent=param_extraction_agent,
    verbose=True
)

generation_task = Task(
    description="Generates protein sequences based on extracted parameters.",
    expected_output="A list of generated protein sequences.",
    tools=[generate_tool],
    agent=generation_agent,
    verbose=True
)

In [121]:
# Initialize Crew
crew = Crew(
    agents=[param_extraction_agent, generation_agent],
    tasks=[param_extraction_task, generation_task],
    verbose=True, 
    process=Process.hierarchical,  # Fix: Use explicit process type
    manager_llm=llm
)



In [122]:
# User Input
# user_input = input("Enter your protein generation request (including a starting sequence if desired): ")
user_input = "Generate a protein sequence starting with MKVSA, with a maximum length of 120 amino acids. Use diverse sampling and ensure low repetition."

In [ ]:
os.environ["HUGGINGFACE_API_KEY"] = "" #put your own huggingface api key here :s 

In [124]:
# Run the Full Workflow
result = crew.kickoff(inputs={"user_prompt": user_input})


19:16:53 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= Kameshr/LLAMA-3-Quantized ; provider = huggingface


 
[2025-03-11 19:16:53][🚀 CREW 'CREW' STARTED, D08D30B3-4278-47A5-AC03-738FA64E4907]: 2025-03-11 19:16:53.367373
 
[2025-03-11 19:16:53][📋 TASK STARTED: EXTRACTS NECESSARY PARAMETERS FROM THE USER INPUT.]: 2025-03-11 19:16:53.397960
 
[2025-03-11 19:16:53][🤖 AGENT 'CREW MANAGER' STARTED TASK]: 2025-03-11 19:16:53.399515
# Agent: Crew Manager
## Task: Extracts necessary parameters from the user input.
 
[2025-03-11 19:16:53][🤖 LLM CALL STARTED]: 2025-03-11 19:16:53.400296


LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 
[2025-03-11 19:16:54][❌ LLM CALL FAILED: 'LITELLM.SERVICEUNAVAILABLEERROR: HUGGINGFACEEXCEPTION - <!DOCTYPE HTML>
<HTML CLASS="" LANG="EN">
<HEAD>
    <META CHARSET="UTF-8" />
    <META
            NAME="VIEWPORT"
            CONTENT="WIDTH=DEVICE-WIDTH, INITIAL-SCALE=1.0, USER-SCALABLE=NO"
    />
    <META
            NAME="DESCRIPTION"
            CONTENT="WE'RE ON A JOURNEY TO A

ServiceUnavailableError: litellm.ServiceUnavailableError: HuggingfaceException - <!DOCTYPE html>
<html class="" lang="en">
<head>
    <meta charset="utf-8" />
    <meta
            name="viewport"
            content="width=device-width, initial-scale=1.0, user-scalable=no"
    />
    <meta
            name="description"
            content="We're on a journey to advance and democratize artificial intelligence through open source and open science."
    />
    <meta property="fb:app_id" content="1321688464574422" />
    <meta name="twitter:card" content="summary_large_image" />
    <meta name="twitter:site" content="@huggingface" />
    <meta
            property="og:title"
            content="Hugging Face - The AI community building the future."
    />
    <meta property="og:type" content="website" />

    <title>Hugging Face - The AI community building the future.</title>
    <style>
        body {
            margin: 0;
        }

        main {
            background-color: white;


In [ ]:
user_test_input = "Generate a protein sequence starting with MKVSA, max length 210, top_k 899."

# Call parameter extraction manually
extracted_params = extract_parameters(user_test_input)

print(extracted_params)
